In [1]:
from narrative_llm_agent.crews.job_crew import JobCrew
from langchain_openai import ChatOpenAI, OpenAI
import openai
import os
from crewai import Crew, Agent, Task
from narrative_llm_agent.agents.kbase_agent import KBaseAgent
from langchain.tools import tool
import json
from pydantic import BaseModel

In [2]:
# used_llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
used_llm = ChatOpenAI(
    model="openai/gpt-4o",
    temperature=0,
    api_key=os.environ.get('CBORG_API_KEY'),
    base_url="https://api.cborg.lbl.gov"  # For LBL-Net, use "https://api-local.cborg.lbl.gov"
)
# used_llm = ChatOpenAI(model="o3-mini", api_key=os.environ.get('CBORG_API_KEY'), base_url="https://api.cborg.lbl.gov")

In [3]:
class AppRunInputs(BaseModel):
    narrative_id: int
    app_id: str
    input_object_upa: str

class WorkflowRunner(KBaseAgent):
    job_crew: JobCrew
    role: str = "You are a workflow runner, your role is to efficiently run KBase workflows."
    goal: str = "Your goal is to create and run elegant and scientifically meaningful computational biology workflows."
    backstory: str = "You are a dedicated and effective computational biologist. You have deep knowledge of how to run workflows in the DOE KBase system and have years of experience using this to produce high quality scientific knowledge."
    
    def __init__(self, llm, token: str = None):
        self.job_crew = JobCrew(llm)
        self._llm = llm
        self._token = token

        @tool(args_schema=AppRunInputs)
        def do_app_run(narrative_id: int, app_id: str, input_object_upa: str):
            """
            This invokes a CrewAI crew to run a new KBase app from start to finish and
            returns the results. It takes in the narrative_id, app_id (formalized as module_name/app_name), and
            UPA of the input object.
            """
            return self.run_app_crew(narrative_id, app_id, input_object_upa)
            
        self.agent = Agent(
            role=self.role,
            goal=self.goal,
            backstory=self.backstory,
            verbose=True,
            tools=[
                do_app_run
            ],  # + human_tools,
            llm=self._llm,
            allow_delegation=False,
            memory=True,
        )
    
    def run_app_crew(self, narrative_id: int, app_id: str, input_object_upa: str):
        return self.job_crew.start_job(app_id, input_object_upa, narrative_id, app_id=app_id)

wf_runner = WorkflowRunner(used_llm)

In [4]:
narrative_id = 77754
reads_id = "77754/2/1"

apps_to_run = [
    {"Step": 1, "Name": "Quality Control", "Description": "Trim low-quality bases and remove adapter sequences from the paired-end reads using Trimmomatic.", "App": "kb_trimmomatic/run_trimmomatic"},
    {"Step": 2, "Name": "Error Correction", "Description": "Correct errors in short Illumina reads using BFC to enhance the quality of reads before assembly.", "App": "kb_bfc/run_bfc"},
    {"Step": 3, "Name": "Genome Assembly", "Description": "Perform de novo assembly of the trimmed reads into contigs and scaffolds using SPAdes.", "App": "kb_SPAdes/run_SPAdes"},
    {"Step": 4, "Name": "Assembly Quality Assessment", "Description": "Obtain assembly statistics such as N50, total length, and number of contigs using QUAST.", "App": "QUAST"},
    {"Step": 5, "Name": "Genome Annotation", "Description": "Annotate the assembled genome using Prokka to provide insights into gene content and functional elements.", "App": "Prokka"},
    {"Step": 6, "Name": "Genome Completeness Estimation", "Description": "Estimate genome completeness and contamination levels using CheckM.", "App": "kb_Msuite/run_checkM_lineage_wf"},
    {"Step": 7, "Name": "Taxonomic Classification", "Description": "Perform taxonomic classification to confirm organism identity using GTDB-Tk.", "App": "kb_gtdbtk/run_kb_gtdbtk_classify_wf"},
    {"Step": 8, "Name": "Documentation and Reporting", "Description": "Document each step, prepare visualizations and tables summarizing assembly statistics, annotations, and taxonomic classifications for inclusion in MRA paper.", "App": ""}
]

steps_to_run = [
    {
        "Step": 1,
        "Name": "Quality control",
        "Description": "Check reads for quality using FastQC",
        "app_id": "kb_fastqc/runFastQC"
    }, 
    {
        "Step": 2,
        "Name": "Reads trimming",
        "Description": "Trim low-quality bases and remove adapter sequences from the paired-end reads using Trimmomatic",
        "app_id": "kb_trimmomatic/run_trimmomatic"
    },
    {
        "Step": 3,
        "Name": "Genome Assembly",
        "Description": "Perform de novo assembly of the trimmed reads into contigs and scaffolds using SPAdes.",
        "app_id": "kb_SPAdes/run_SPAdes"
    },
]

run_apps_task = Task(
    description=f"""This task involves running multiple apps where the output of one (if any) is fed into the next as input. Here are the tasks in JSON format: {json.dumps(steps_to_run)}. These steps must be run sequentially. These must be run in the narrative with id {narrative_id} and start with using the paired-end reads object {reads_id}. In the end, return a brief summary of steps taken and resulting output objects.""",
    expected_output="A summary of task completion, the number of apps run, and the upa of any output objects.",
    agent=wf_runner.agent
)

In [5]:
crew = Crew(
    agents = [wf_runner.agent],
    tasks=[run_apps_task],
    verbose=True,
)
crew.kickoff()

# Agent: You are a workflow runner, your role is to efficiently run KBase workflows.
## Task: This task involves running multiple apps where the output of one (if any) is fed into the next as input. Here are the tasks in JSON format: [{"Step": 2, "Name": "Reads trimming", "Description": "Trim low-quality bases and remove adapter sequences from the paired-end reads using Trimmomatic", "app_id": "kb_trimmomatic/run_trimmomatic"}]. These steps must be run sequentially. These must be run in the narrative with id 77754 and start with using the paired-end reads object 77754/2/1. In the end, return a brief summary of steps taken and resulting output objects.


Overriding of current TracerProvider is not allowed


# Agent: Project coordinator
## Task: 
            From the given KBase app id, kb_trimmomatic/run_trimmomatic, fetch the list of parameters needed to run it. Use the App and Job manager agent
            for assistance. With the knowledge that there is a data object with id "77754/2/1", populate a dictionary
            with the parameters where the keys are parameter ids, and values are the proper parameter values, or their
            default values if no value can be found or calculated. Be sure to make sure there is a non-null value for any parameter that is not optional.
            Any parameter that has a true value for "is_output_object" must have a valid name for the new object. The new object name should be based on
            the input object name, not its upa. Only alphanumeric characters and underscores are allowed in new object names.
            Return the dictionary of inputs, the app id, and the
            narrative id 77754 for use in the next task. Do not add comm

CrewOutput(raw='The "Reads trimming" app was successfully run in the narrative with ID 77754. A total of 1 app was executed. The output object UPA from this step is not specified in the observation, but the task was completed successfully.\n```', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='This task involves running multiple apps where the output of one (if any) is fed into the next as input. Here are the tasks in JSON format: [{"Step": 2, "Name": "Reads trimming", "Description": "Trim low-quality bases and remove adapter sequences from the paired-end reads using Trimmomatic", "app_id": "kb_trimmomatic/run_trimmomatic"}]. These steps must be run sequentially. These must be run in the narrative with id 77754 and start with using the paired-end reads object 77754/2/1. In the end, return a brief summary of steps taken and resulting output objects.', name=None, expected_output='A summary of task completion, the number of apps run, and the upa of any output objects.

In [5]:
crew = Crew(
    agents = [wf_runner.agent],
    tasks=[run_apps_task],
    verbose=True,
)
crew.kickoff()

# Agent: You are a workflow runner, your role is to efficiently run KBase workflows.
## Task: This task involves running multiple apps where the output of one (if any) is fed into the next as input. Here are the tasks in JSON format: [{"Step": 2, "Name": "Reads trimming", "Description": "Trim low-quality bases and remove adapter sequences from the paired-end reads using Trimmomatic", "app_id": "kb_trimmomatic/run_trimmomatic"}]. These steps must be run sequentially. These must be run in the narrative with id 77754 and start with using the paired-end reads object 77754/2/1. In the end, return a brief summary of steps taken and resulting output objects.


Overriding of current TracerProvider is not allowed


# Agent: Project coordinator
## Task: 
            From the given KBase app id, kb_trimmomatic/run_trimmomatic, fetch the list of parameters needed to run it. Use the App and Job manager agent
            for assistance. With the knowledge that there is a data object with id "77754/2/1", populate a dictionary
            with the parameters where the keys are parameter ids, and values are the proper parameter values, or their
            default values if no value can be found or calculated. Be sure to make sure there is a non-null value for any parameter that is not optional.
            Any parameter that has a true value for "is_output_object" must have a valid name for the new object. The new object name should be based on
            the input object name, not its upa. Only alphanumeric characters and underscores are allowed in new object names.
            Return the dictionary of inputs, the app id, and the
            narrative id 77754 for use in the next task. Do not add comm

⚠️ It looks like you upgraded from a version below 0.5.6 and could benefit from vacuuming your database. Run chromadb utils vacuum --help for more information.
/Users/wjriehl/Projects/kbase/narrative_llm_agent/narrative_llm_agent/agents/analyst.py:175: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")




# Agent: KBase Analyst and Information Provider
## Thought: To find the name of the data object with the specified ID, I need to search the KBase documentation for information on how to retrieve or identify data object names using their IDs.
## Using tool: KBase documentation retrieval tool
## Tool Input: 
"{\"query\": \"How to find the name of a data object using its ID in KBase?\"}"
## Tool Output: 
{'query': 'How to find the name of a data object using its ID in KBase?', 'chat_history': '', 'result': 'To find the name of a data object using its ID in KBase, you can follow these steps:\n\n1. **Log in to KBase**: Access your KBase account and navigate to the Narrative Interface.\n\n2. **Locate the Data Object**:\n   - Go to the **Data Panel** under the **Analyze** tab.\n   - Use the search functionality to search for the data object using its ID. You can enter the ID in the search bar to locate the specific data object.\n\n3. **View Data Object Details**:\n   - Once you locate the d

CrewOutput(raw='The task involved running one app, "Reads trimming" using Trimmomatic, in narrative 77754. The input object was 77754/2/1. The app was successfully executed, resulting in the output object \'Sample_Reads_trimmed\' with UPA 77754/15/1.\n```', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='This task involves running multiple apps where the output of one (if any) is fed into the next as input. Here are the tasks in JSON format: [{"Step": 2, "Name": "Reads trimming", "Description": "Trim low-quality bases and remove adapter sequences from the paired-end reads using Trimmomatic", "app_id": "kb_trimmomatic/run_trimmomatic"}]. These steps must be run sequentially. These must be run in the narrative with id 77754 and start with using the paired-end reads object 77754/2/1. In the end, return a brief summary of steps taken and resulting output objects.', name=None, expected_output='A summary of task completion, the number of apps run, and the upa of any outp

In [5]:
crew = Crew(
    agents = [wf_runner.agent],
    tasks=[run_apps_task],
    verbose=True,
)
crew.kickoff()

# Agent: You are a workflow runner, your role is to efficiently run KBase workflows.
## Task: This task involves running multiple apps where the output of one (if any) is fed into the next as input. Here are the tasks in JSON format: [{"Step": 1, "Name": "Quality control", "Description": "Check reads for quality using FastQC", "app_id": "kb_fastqc/runFastQC"}, {"Step": 2, "Name": "Reads trimming", "Description": "Trim low-quality bases and remove adapter sequences from the paired-end reads using Trimmomatic", "app_id": "kb_trimmomatic/run_trimmomatic"}, {"Step": 3, "Name": "Genome Assembly", "Description": "Perform de novo assembly of the trimmed reads into contigs and scaffolds using SPAdes.", "app_id": "kb_SPAdes/run_SPAdes"}]. These steps must be run sequentially. These must be run in the narrative with id 77754 and start with using the paired-end reads object 77754/2/1. In the end, return a brief summary of steps taken and resulting output objects.


Overriding of current TracerProvider is not allowed


# Agent: Project coordinator
## Task: 
            From the given KBase app id, kb_fastqc/runFastQC, fetch the list of parameters needed to run it. Use the App and Job manager agent
            for assistance. With the knowledge that there is a data object with id "77754/2/1", populate a dictionary
            with the parameters where the keys are parameter ids, and values are the proper parameter values, or their
            default values if no value can be found or calculated. Be sure to make sure there is a non-null value for any parameter that is not optional.
            Any parameter that has a true value for "is_output_object" must have a valid name for the new object. The new object name should be based on
            the input object name, not its upa. Only alphanumeric characters and underscores are allowed in new object names.
            Return the dictionary of inputs, the app id, and the
            narrative id 77754 for use in the next task. Do not add comments or oth

Overriding of current TracerProvider is not allowed


# Agent: Project coordinator
## Task: 
            From the given KBase app id, kb_trimmomatic/run_trimmomatic, fetch the list of parameters needed to run it. Use the App and Job manager agent
            for assistance. With the knowledge that there is a data object with id "77754/2/1", populate a dictionary
            with the parameters where the keys are parameter ids, and values are the proper parameter values, or their
            default values if no value can be found or calculated. Be sure to make sure there is a non-null value for any parameter that is not optional.
            Any parameter that has a true value for "is_output_object" must have a valid name for the new object. The new object name should be based on
            the input object name, not its upa. Only alphanumeric characters and underscores are allowed in new object names.
            Return the dictionary of inputs, the app id, and the
            narrative id 77754 for use in the next task. Do not add comm

...

# Agent: Project coordinator
## Task: 
            From the given KBase app id, kb_SPAdes/run_SPAdes, fetch the list of parameters needed to run it. Use the App and Job manager agent
            for assistance. With the knowledge that there is a data object with id "77754/17/1", populate a dictionary
            with the parameters where the keys are parameter ids, and values are the proper parameter values, or their
            default values if no value can be found or calculated. Be sure to make sure there is a non-null value for any parameter that is not optional.
            Any parameter that has a true value for "is_output_object" must have a valid name for the new object. The new object name should be based on
            the input object name, not its upa. Only alphanumeric characters and underscores are allowed in new object names.
            Return the dictionary of inputs, the app id, and the
            narrative id 77754 for use in the next task. Do not add comments or o

ERROR:root:LiteLLM call failed: litellm.Timeout: APITimeoutError - Request timed out. Error_str: Request timed out.




LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

 Error during LLM call: litellm.Timeout: APITimeoutError - Request timed out. Error_str: Request timed out.
# Agent: Project coordinator
## Task: 
            From the given KBase app id, kb_SPAdes/run_SPAdes, fetch the list of parameters needed to run it. Use the App and Job manager agent
            for assistance. With the knowledge that there is a data object with id "77754/17/1", populate a dictionary
            with the parameters where the keys are parameter ids, and values are the proper parameter values, or their
            default values if no value can be found or calculated. Be sure to make sure there is a non-null value for any parameter that is not optional.
            Any parameter that has a true value for "is_output_object" must have a valid name for the new object. The new object name should be based on
            the input object name, not its upa. Only alphanumeric characters and u

# Agent: Project coordinator
## Task: 
            From the given KBase app id, kb_SPAdes/run_SPAdes, fetch the list of parameters needed to run it. Use the App and Job manager agent
            for assistance. With the knowledge that there is a data object with id "77754/2/1", populate a dictionary
            with the parameters where the keys are parameter ids, and values are the proper parameter values, or their
            default values if no value can be found or calculated. Be sure to make sure there is a non-null value for any parameter that is not optional.
            Any parameter that has a true value for "is_output_object" must have a valid name for the new object. The new object name should be based on
            the input object name, not its upa. Only alphanumeric characters and underscores are allowed in new object names.
            Return the dictionary of inputs, the app id, and the
            narrative id 77754 for use in the next task. Do not add comments or ot

CrewOutput(raw="The workflow was completed successfully with the following steps:\n1. Quality control using FastQC was run with input UPA 77754/2/1, resulting in report UPA 77754/16/1.\n2. Reads trimming using Trimmomatic was run with input UPA 77754/2/1, resulting in report UPA 77754/17/1.\n3. Genome Assembly using SPAdes was run with input UPA 77754/2/1, resulting in output object 'SampleDataObject_Assembly' and report UPA 77754/19/1.\n\nA total of 3 apps were run, and the final output object is 'SampleDataObject_Assembly'.\n```", pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='This task involves running multiple apps where the output of one (if any) is fed into the next as input. Here are the tasks in JSON format: [{"Step": 1, "Name": "Quality control", "Description": "Check reads for quality using FastQC", "app_id": "kb_fastqc/runFastQC"}, {"Step": 2, "Name": "Reads trimming", "Description": "Trim low-quality bases and remove adapter sequences from the paired-e